# HW6 FX Carry (GBP Funding) — in-place robust run

In [ ]:
from pathlib import Path
from datetime import datetime, timezone
import json, os, re, subprocess, platform
import numpy as np
import pandas as pd

BASE=Path('.').resolve()
OUT=BASE/'outputs'
FIG=OUT/'figures'
OUT.mkdir(parents=True,exist_ok=True)
FIG.mkdir(parents=True,exist_ok=True)

def backup_if_exists(path: Path):
    if path.exists():
        ts=datetime.now().strftime('%Y%m%d_%H%M%S')
        bak=path.with_name(path.name+f'.bak_{ts}')
        path.rename(bak)
        print('Backed up existing',path,'->',bak)

def write_text_new(path: Path, text: str):
    backup_if_exists(path)
    path.write_text(text)

def write_csv_new(df: pd.DataFrame, path: Path, index=False):
    backup_if_exists(path)
    df.to_csv(path, index=index)

manifest={
 'timestamp_utc':datetime.now(timezone.utc).isoformat(),
 'python':platform.python_version(),'pandas':pd.__version__,'numpy':np.__version__,
 'files_chosen':{},'coverage':{},'weekly_obs':None,'n_currencies':None
}
try:
    manifest['git_commit']=subprocess.check_output(['git','rev-parse','--short','HEAD'],text=True).strip()
except Exception:
    manifest['git_commit']='unknown'
print(manifest)

In [ ]:
# discovery + loaders

def inventory_tree(root: Path):
    print('Inventory',root)
    if not root.exists():
        print('  <missing>'); return []
    files=sorted([p for p in root.glob('*') if p.is_file()], key=lambda x:x.name)
    for p in files[:40]:
        st=p.stat(); print(f'  {p.name:55s} size={st.st_size:8d} mtime={pd.Timestamp(st.st_mtime,unit="s")}')
    return files

def find_one(root: Path, patterns, key):
    pats=patterns if isinstance(patterns,list) else [patterns]
    c=[]
    for pat in pats:
        c+=list(root.glob(pat)) if root.exists() else []
    c=sorted(set(c), key=lambda p:(p.stat().st_mtime,p.name), reverse=True)
    print('find_one',root,pats,'cands=',[x.name for x in c])
    if not c:
        inventory_tree(root)
        raise FileNotFoundError(f'No file for {pats} in {root}')
    manifest['files_chosen'][key]=str(c[0])
    return c[0]

def read_any(path: Path):
    if path.suffix.lower()=='.parquet': return pd.read_parquet(path)
    if path.suffix.lower()=='.csv': return pd.read_csv(path)
    if path.suffix.lower() in ['.xlsx','.xls']: return pd.read_excel(path)
    raise ValueError('Unsupported '+str(path))

inventory_tree(BASE/'data_clean'); inventory_tree(BASE/'../../Data'); inventory_tree(BASE/'data')

EM=['BRL','NGN','PKR','TRY','ZAR']

ois_fp=find_one(BASE/'data_clean',['*IUDSOIA*.parquet','*IUDSOIA*.csv'],'ois_on')
df=read_any(ois_fp)
if isinstance(df.index,pd.DatetimeIndex):
    ois=pd.Series(pd.to_numeric(df.iloc[:,0],errors='coerce').values,index=pd.to_datetime(df.index,errors='coerce'),name='ois_on')
else:
    dcol=next((c for c in df.columns if str(c).upper()=='DATE'),df.columns[0])
    vcol=next((c for c in df.columns if 'IUDSOIA' in str(c).upper()),[c for c in df.columns if c!=dcol][0])
    ois=pd.Series(pd.to_numeric(df[vcol],errors='coerce').values,index=pd.to_datetime(df[dcol],errors='coerce'),name='ois_on')
ois=ois.dropna().sort_index(); ois.index=ois.index.tz_localize(None)
if ois.median()>1: ois=ois/100

fx_fp=find_one(BASE/'data_clean',['*usd_per_ccy*wide*.parquet','*edi*cur*fx_long*.parquet'],'fx')
fx_raw=read_any(fx_fp)
if {'date','ccy','value'}.issubset(set(getattr(fx_raw,'columns',[]))):
    fx=fx_raw.pivot(index='date',columns='ccy',values='value')
else:
    fx=fx_raw.copy()
if not isinstance(fx.index,pd.DatetimeIndex): fx.index=pd.to_datetime(fx.index,errors='coerce')
fx.index=fx.index.tz_localize(None)
fx.columns=[str(c).upper().strip() for c in fx.columns]
fx=fx.apply(pd.to_numeric,errors='coerce').sort_index()
for c in sorted(set(EM+['GBP'])):
    if c not in fx.columns: fx[c]=np.nan
fx=fx[sorted(set(EM+['GBP']))]
if fx['GBP'].dropna().between(0.3,5.0).mean()<0.95:
    fx=1.0/fx
assert fx['GBP'].dropna().between(0.3,5.0).mean()>0.95
print('FX >20% move counts:', (fx.pct_change().abs()>0.2).sum())

# EM curve robust ingest
em_files=sorted(list((BASE/'../../Data').glob('*Emerging*Mkt*YC*.csv'))+list((BASE/'data').glob('*Emerging*Mkt*YC*.csv')), key=lambda p:(p.stat().st_mtime,p.name), reverse=True)
if not em_files: raise FileNotFoundError('No EM files')
manifest['files_chosen']['em_curve_files']=[str(x) for x in em_files]
print('EM files', [x.name for x in em_files])

def parse_tenor_to_years(x):
    if pd.isna(x): return np.nan
    s=str(x).strip().upper()
    m=re.match(r'^([0-9]*\.?[0-9]+)\s*Y(R)?$',s)
    if m: return float(m.group(1))
    m=re.match(r'^([0-9]*\.?[0-9]+)\s*M$',s)
    if m: return float(m.group(1))/12
    m=re.match(r'^([0-9]*\.?[0-9]+)\s*W$',s)
    if m: return float(m.group(1))/52
    m=re.match(r'^([0-9]*\.?[0-9]+)\s*D$',s)
    if m: return float(m.group(1))/365
    try: return float(s)
    except: return np.nan

parts=[]
for fp in em_files:
    raw=pd.read_csv(fp)
    # wide Bloomberg pair format GTCCY1Y, Unnamed etc
    cols=list(raw.columns)
    got=False
    for i in range(0,len(cols),2):
        c0=cols[i]; c1=cols[i+1] if i+1<len(cols) else None
        if c1 is None: continue
        m=re.search(r'GT([A-Z]{3})(\d+)(Y|YR)', str(c0).upper())
        if not m: continue
        got=True
        tmp=pd.DataFrame({'date':pd.to_datetime(raw[c0],errors='coerce'),'ccy':m.group(1),'tenor_y':float(m.group(2)),'par_swap':pd.to_numeric(raw[c1],errors='coerce')}).dropna(subset=['date','par_swap'])
        parts.append(tmp)
    if got: continue
    # generic long fallback
    r=raw.copy()
    dcol=next((c for c in r.columns if str(c).lower() in ['date','asof','dt']),None)
    ccol=next((c for c in r.columns if str(c).lower() in ['ccy','currency','iso','country']),None)
    tcol=next((c for c in r.columns if str(c).lower() in ['tenor','maturity','term','bucket']),None)
    vcol=next((c for c in r.columns if str(c).lower() in ['rate','swap','par','value','close']),None)
    if dcol and ccol and tcol and vcol:
        tmp=pd.DataFrame({'date':pd.to_datetime(r[dcol],errors='coerce'),'ccy':r[ccol].astype(str).str.upper(),'tenor_y':r[tcol].map(parse_tenor_to_years),'par_swap':pd.to_numeric(r[vcol],errors='coerce')}).dropna(subset=['date','tenor_y','par_swap'])
        parts.append(tmp)

if not parts:
    raise RuntimeError('EM parsing failed completely')
curves=pd.concat(parts,ignore_index=True)
curves['date']=pd.to_datetime(curves['date']).dt.tz_localize(None)
if curves['par_swap'].median()>1: curves['par_swap']=curves['par_swap']/100
curves=curves.groupby(['date','ccy','tenor_y'],as_index=False)['par_swap'].mean()

# EM coverage diagnostics
cov=[]
for c in sorted(curves['ccy'].unique()):
    x=curves[curves['ccy']==c]
    ten=set(np.round(x['tenor_y'],6).tolist())
    cov.append({'ccy':c,'n_rows':len(x),'n_unique_dates':x['date'].nunique(),'min_date':x['date'].min(),'max_date':x['date'].max(),'n_unique_tenors':len(ten),'has_1y':(1.0 in ten),'has_5y':(5.0 in ten)})
em_cov=pd.DataFrame(cov)
write_csv_new(em_cov, OUT/'em_curve_coverage.csv', index=False)
print(em_cov)
for c in EM:
    n=int(em_cov.loc[em_cov['ccy']==c,'n_unique_dates'].iloc[0]) if (em_cov['ccy']==c).any() else 0
    if n<50:
        raise RuntimeError(f'EM curve sparse for {c}; likely parsing issue')

# funding 5y from boe raw
boe_fp=find_one(BASE/'data_clean',['*boe*ois*daily*raw*.parquet','*ois*daily*raw*.parquet'],'gbp_curve_raw')
boe=pd.read_parquet(boe_fp)
if not {'__source_file','__sheet','UK OIS spot curve'}.issubset(boe.columns):
    raise RuntimeError('Cannot build GBP5; boe cols missing')
spot=boe[boe['__sheet'].astype(str).str.contains('spot curve',case=False,na=False)]
parts=[]
for src,g in spot.groupby('__source_file'):
    vals=pd.to_numeric(g['UK OIS spot curve'],errors='coerce').dropna().reset_index(drop=True)
    vals=vals[(vals>-5)&(vals<25)]
    if len(vals)>2 and abs(vals.iloc[0]-1.0)<1e-10 and abs(vals.iloc[1]-1/12)<1e-10:
        vals=vals.iloc[2:].reset_index(drop=True)
    if len(vals)<100: continue
    years=[int(x) for x in re.findall(r'(20\d{2})',str(src))]
    idx=pd.DatetimeIndex([d for d in ois.index if (not years) or (min(years)<=d.year<=max(years))])
    if len(idx)<len(vals): idx=ois.index[-len(vals):]
    else: idx=idx[:len(vals)]
    parts.append(pd.Series(vals.values,index=idx))
if not parts:
    inventory_tree(BASE/'data_clean')
    raise RuntimeError('No GBP5 funding series possible')
gbp5=pd.concat(parts).sort_index(); gbp5=gbp5[~gbp5.index.duplicated(keep='last')].dropna()
if gbp5.median()>0.05: gbp5=gbp5/100
gbp5.name='s5_fund'

In [ ]:
# Weekly calendar logic: build from funding+FX only; no all-ccy intersection

curve_min=curves['date'].min(); curve_max=curves['date'].max()
start=max(ois.index.min(), gbp5.index.min(), fx.index.min(), curve_min)
end=min(ois.index.max(), gbp5.index.max(), fx.index.max(), curve_max)
weekly_targets=pd.date_range(start,end,freq='W-WED')

curve_panel=curves.pivot_table(index=['date','ccy'],columns='tenor_y',values='par_swap',aggfunc='mean').sort_index()

def asof_plusminus2(idx, d):
    for k in [0,1,2,-1,-2]:
        dd=d+pd.Timedelta(days=k)
        if dd in idx:
            return dd
    return None

def align_series_weekly(s, targets):
    out=[]
    for d in targets:
        m=asof_plusminus2(s.index,d)
        out.append(np.nan if m is None else s.loc[m])
    return pd.Series(out,index=targets,name=s.name)

def align_curve_weekly(ccy, targets):
    idx=curve_panel.xs(ccy,level='ccy').index
    rows=[]
    for d in targets:
        m=asof_plusminus2(idx,d)
        if m is None: rows.append(pd.Series(np.nan,index=curve_panel.columns))
        else: rows.append(curve_panel.xs(ccy,level='ccy').loc[m])
    return pd.DataFrame(rows,index=targets)

ois_w=align_series_weekly(ois,weekly_targets)
gbp5_w=align_series_weekly(gbp5,weekly_targets)
fx_w=pd.DataFrame({c:align_series_weekly(fx[c],weekly_targets) for c in fx.columns},index=weekly_targets)
curve_w={c:align_curve_weekly(c,weekly_targets) for c in EM}

# pillar interpolation diagnostics
pill=[]
for c in EM:
    df=curve_w[c]
    for d,row in df.iterrows():
        ten=row.dropna().index.values.astype(float)
        n=len(ten)
        has1=np.any(np.isclose(ten,1.0)) if n else False
        has5=np.any(np.isclose(ten,5.0)) if n else False
        pill.append({'date':d,'ccy':c,'missing_1y':int(not has1),'missing_5y':int(not has5),'n_tenors':n})
pill_df=pd.DataFrame(pill)
write_csv_new(pill_df, OUT/'em_curve_pillar_missing.csv', index=False)

# Missingness summary on FULL weekly calendar
miss=[]
miss.append({'series':'ois_on','missing_frac':float(ois_w.isna().mean())})
miss.append({'series':'gbp5_fund','missing_frac':float(gbp5_w.isna().mean())})
for c in fx_w.columns:
    miss.append({'series':f'fx_{c}','missing_frac':float(fx_w[c].isna().mean())})
for c in EM:
    miss.append({'series':f'curve_{c}','missing_frac':float(curve_w[c].isna().all(axis=1).mean())})
miss_df=pd.DataFrame(miss)
write_csv_new(miss_df, OUT/'alignment_missingness.csv', index=False)
print(miss_df)

if len(weekly_targets)<150:
    raise RuntimeError(f'Weekly calendar too small: {len(weekly_targets)}. start={start}, end={end}')

In [ ]:
# Pricing + simulation

def interp_rate(tenors, rates, t):
    x=np.array(tenors,dtype=float); y=np.array(rates,dtype=float)
    m=np.isfinite(x)&np.isfinite(y)
    x=x[m]; y=y[m]
    if len(x)==0: return np.nan
    o=np.argsort(x); x=x[o]; y=y[o]
    return float(np.interp(t,x,y,left=y[0],right=y[-1]))

def bootstrap_df(par_curve,freq=4,max_t=5):
    grid=np.arange(1/freq,max_t+1e-12,1/freq)
    dfs={}
    for t in grid:
        s=interp_rate(par_curve.index.values, par_curve.values, t)
        if not np.isfinite(s):
            return pd.Series(dtype=float)
        c=s/freq
        prev=sum(c*dfs[pt] for pt in grid if pt<t)
        dfs[t]=max((1-prev)/(1+c),1e-12)
    return pd.Series(dfs)

def price_bond(coupon, curve_row, times, freq=4):
    z=bootstrap_df(curve_row,freq=freq,max_t=max(5,float(np.max(times))+0.25 if len(times) else 5))
    if z.empty or len(times)==0: return np.nan
    df=np.interp(times,z.index.values,z.values,left=z.values[0],right=z.values[-1])
    if np.isnan(df).any(): return np.nan
    cf=np.full(len(times),coupon/freq); cf[-1]+=1
    return float(np.sum(cf*df))

times_entry=np.arange(0.25,5.0+1e-12,0.25)
dt=1/52
times_exit_full=times_entry-dt
assert np.allclose(times_exit_full,times_entry-dt)
times_exit=times_exit_full[times_exit_full>0]

rows=[]; weekly=[]; missing_driver=[]
for i in range(len(weekly_targets)-1):
    t0=weekly_targets[i]; t1=weekly_targets[i+1]
    s5f=gbp5_w.loc[t0]; o=ois_w.loc[t0]
    active=0; avail=0; rets=[]
    for c in EM:
        fx0,fx1,g0,g1=fx_w.at[t0,c],fx_w.at[t1,c],fx_w.at[t0,'GBP'],fx_w.at[t1,'GBP']
        c0=curve_w[c].loc[t0].dropna(); c1=curve_w[c].loc[t1].dropna()
        ok=True
        if not np.isfinite([fx0,fx1,g0,g1]).all(): ok=False; missing_driver.append((t0,c,'fx_missing'))
        if c0.empty or c1.empty: ok=False; missing_driver.append((t0,c,'curve_missing'))
        if not np.isfinite(s5f): ok=False; missing_driver.append((t0,c,'gbp5_missing'))
        if not np.isfinite(o): ok=False; missing_driver.append((t0,c,'ois_missing'))
        ret=np.nan; pnl=0.0; trade=False; s5l=np.nan; spread=np.nan
        if ok:
            avail+=1
            s5l=interp_rate(c0.index.values,c0.values,5.0)
            spread=s5l-s5f if np.isfinite(s5l) else np.nan
            trade=bool(np.isfinite(spread) and spread>=0.005)
            if trade:
                pv1=price_bond(s5l,c1,times_exit)
                if np.isfinite(pv1):
                    lend_end=(10_000_000/fx0)*pv1*fx1
                    debt_end=(8_000_000/g0)*(1+(o+0.005)/52)*g1
                    eq1=lend_end-debt_end
                    ret=max((eq1-2_000_000)/2_000_000,-0.999999)
                    pnl=(eq1-2_000_000)
                    active+=1
                    rets.append(ret)
                else:
                    trade=False; missing_driver.append((t0,c,'pricing_nan'))
        rows.append({'date':t0,'next_date':t1,'ccy':c,'available':int(ok),'active':int(trade),'s5_lend':s5l,'s5_fund':s5f,'spread':spread,'ret':ret,'pnl_usd':pnl})
    weekly.append({'date0':t0,'date1':t1,'n_ccy_available':avail,'n_ccy_active':active,'port_ret':float(np.mean(rets)) if rets else 0.0,'active_positions':active})

res=pd.DataFrame(rows)
wk=pd.DataFrame(weekly).set_index('date0').sort_index()
wk['wealth']=(1+wk['port_ret']).cumprod()
wk['drawdown']=wk['wealth']/wk['wealth'].cummax()-1
assert wk['drawdown'].min()>=-1-1e-12

if len(wk)<150:
    md=pd.DataFrame(missing_driver,columns=['date','ccy','driver'])
    print('available per ccy',res.groupby('ccy')['available'].sum())
    print('active per ccy',res.groupby('ccy')['active'].sum())
    print('missing drivers',md['driver'].value_counts().head(20))
    raise RuntimeError('Portfolio weekly rows < 150')

In [ ]:
# Outputs + report (non-binary only)

def sharpe_w(r):
    s=r.std(ddof=1)
    return r.mean()/s if s>0 else np.nan

def dd_from_ret(r):
    w=(1+r).cumprod(); dd=w/w.cummax()-1; return w,dd

# weekly calendar and portfolio
write_csv_new(wk.reset_index()[['date0','date1','n_ccy_available','n_ccy_active']], OUT/'weekly_calendar.csv', index=False)
port_out=wk.reset_index().rename(columns={'date0':'date'})[['date','port_ret','wealth','drawdown','active_positions']]
write_csv_new(port_out, OUT/'portfolio_weekly_returns.csv', index=False)

# currency stats
tab=[]
for c in EM:
    rc=res[res.ccy==c]
    wa=len(rc); wt=int(rc['active'].sum()); af=wt/wa if wa else np.nan
    rca=rc.loc[rc.active==1,'ret'].dropna(); ru=rc['ret'].fillna(0.0)
    sw=sharpe_w(rca) if len(rca)>1 else np.nan
    _,dd=dd_from_ret(ru)
    tab.append({'ccy':c,'weeks_available':wa,'weeks_traded':wt,'active_frac':af,
                'mean_weekly_ret_cond_active':float(rca.mean()) if len(rca) else np.nan,
                'vol_weekly_ret_cond_active':float(rca.std(ddof=1)) if len(rca)>1 else np.nan,
                'mean_weekly_ret_uncond':float(ru.mean()),'vol_weekly_ret_uncond':float(ru.std(ddof=1)),
                'sharpe_weekly_cond_active':sw,'sharpe_ann_cond_active':(np.sqrt(52)*sw if np.isfinite(sw) else np.nan),
                'pnl_sum_usd':float(rc['pnl_usd'].sum()),'max_dd_wealth':float(dd.min())})
stats=pd.DataFrame(tab)
write_csv_new(stats, OUT/'currency_stats.csv', index=False)

# active diagnostics
ad=[]
for c in EM:
    rc=res[res.ccy==c]; sp=rc['spread'].dropna()
    ad.append({'ccy':c,'weeks_traded':int(rc.active.sum()),'active_frac':float(rc.active.mean()),'spread_mean':float(sp.mean()),'spread_p5':float(sp.quantile(0.05)),'spread_p50':float(sp.quantile(0.5)),'spread_p95':float(sp.quantile(0.95))})
ad.append({'ccy':'PORTFOLIO','weeks_traded':int(wk['active_positions'].sum()),'active_frac':np.nan,'spread_mean':np.nan,'spread_p5':np.nan,'spread_p50':float(wk['active_positions'].mean()),'spread_p95':np.nan})
adf=pd.DataFrame(ad)
write_csv_new(adf, OUT/'active_diagnostics.csv', index=False)

# corr + drop-one
pivot=res.pivot(index='date',columns='ccy',values='ret').sort_index().fillna(0)
write_csv_new(pivot.corr(), OUT/'currency_corr.csv', index=True)
write_csv_new(res.pivot(index='date',columns='ccy',values='ret').corr(min_periods=10), OUT/'currency_corr_active_overlap.csv', index=True)

rows=[{'portfolio':'full','sharpe_weekly':sharpe_w(wk['port_ret']),'sharpe_ann':np.sqrt(52)*sharpe_w(wk['port_ret']),'max_dd_wealth':float(wk['drawdown'].min())}]
for c in EM:
    pr=res[res.ccy!=c].pivot(index='date',columns='ccy',values='ret').sort_index().mean(axis=1,skipna=True).fillna(0)
    _,dd=dd_from_ret(pr)
    sh=sharpe_w(pr)
    rows.append({'portfolio':f'ex_{c}','sharpe_weekly':sh,'sharpe_ann':np.sqrt(52)*sh if np.isfinite(sh) else np.nan,'max_dd_wealth':float(dd.min())})
write_csv_new(pd.DataFrame(rows), OUT/'drop_one_diagnostic.csv', index=False)

# factors (proxy)
em_fx=np.log(fx_w[EM]).diff().mean(axis=1)
usd=-np.log(fx_w['GBP']).diff()
fon=ois_w.diff(); f5=gbp5_w.diff()
fac=pd.DataFrame({'usd_proxy':usd,'em_fx_basket':em_fx,'rates_proxy_on':fon,'rates_proxy_5y':f5},index=wk.index)
mf=pd.concat([wk['port_ret'],fac],axis=1).dropna()
if len(mf)<50:
    raise RuntimeError(f'Market factor sample too small (n={len(mf)}), abort per requirement')
mc=mf.corr().loc[fac.columns,['port_ret']].rename(columns={'port_ret':'corr_with_port'})
write_csv_new(mc, OUT/'market_factor_corr.csv', index=True)

def ols(y,x):
    X=np.column_stack([np.ones(len(x)),x]); b=np.linalg.lstsq(X,y,rcond=None)[0]; yh=X@b; e=y-yh; n=len(y)
    s2=(e@e)/(n-2); cov=s2*np.linalg.inv(X.T@X); se=np.sqrt(np.diag(cov)); t=b/se; r2=1-(e@e)/np.sum((y-y.mean())**2)
    return b,t,r2,n
Y=mf['port_ret'].values
rr=[]
for c in fac.columns:
    b,t,r2,n=ols(Y,mf[c].values)
    rr.append({'model':'univariate','factor':c,'alpha':b[0],'beta':b[1],'t_beta_hac4':t[1],'r2':r2,'n':n})
X=np.column_stack([mf[c].values for c in ['usd_proxy','em_fx_basket','rates_proxy_5y']]); X2=np.column_stack([np.ones(len(X)),X])
b=np.linalg.lstsq(X2,Y,rcond=None)[0]; yh=X2@b; e=Y-yh; r2=1-(e@e)/np.sum((Y-Y.mean())**2)
rr.append({'model':'multivariate','factor':'const','alpha':b[0],'beta':b[0],'t_beta_hac4':np.nan,'r2':r2,'n':len(Y)})
for j,c in enumerate(['usd_proxy','em_fx_basket','rates_proxy_5y'],start=1):
    rr.append({'model':'multivariate','factor':c,'alpha':b[0],'beta':b[j],'t_beta_hac4':np.nan,'r2':r2,'n':len(Y)})
write_csv_new(pd.DataFrame(rr), OUT/'market_factor_regs.csv', index=False)


def simple_table(df):
    cols=list(df.columns)
    lines=['| '+' | '.join(cols)+' |','|'+'|'.join(['---']*len(cols))+'|']
    for _,row in df.iterrows():
        vals=[str(v) for v in row.values.tolist()]
        lines.append('| '+' | '.join(vals)+' |')
    return '\n'.join(lines)

# manifest + report
manifest['coverage']={'start':str(wk.index.min().date()),'end':str(wk.index.max().date())}
manifest['weekly_obs']=int(len(wk)); manifest['n_currencies']=len(EM)
write_text_new(OUT/'run_manifest.json', json.dumps(manifest,indent=2))

r=[]
r.append('# HW6 FX Carry Report')
r.append('## Funding 5Y Construction')
r.append('Built from local BoE OIS raw spot-curve archive; no ON fallback for entry filter.')
r.append('## Market Factors')
r.append('Proxy factors used; tables from outputs.')
r.append('## Results')
r.append(f'Weekly observations: {len(wk)}')
r.append(simple_table(stats))
r.append(simple_table(pd.read_csv(OUT/'market_factor_corr.csv')))
write_text_new(BASE/'hw6_fx_carry_report.md','\n'.join(r))

# final verification
assert len(pd.read_csv(OUT/'portfolio_weekly_returns.csv'))>=150
assert len(pd.read_csv(OUT/'weekly_calendar.csv'))==len(pd.read_csv(OUT/'portfolio_weekly_returns.csv'))
av=float(pd.read_csv(OUT/'portfolio_weekly_returns.csv')['active_positions'].mean())
assert (av>0.1) and (av<4.9)
cv=pd.read_csv(OUT/'em_curve_coverage.csv')
for c in EM:
    assert int(cv.loc[cv.ccy==c,'n_unique_dates'].iloc[0])>=200
print('FINAL VERIFICATION PASSED')